# SQLite query Practice

SQLite Practice using olist datasets from [Kaggle Olist sample](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce).

This practice utilizes Pandas sql query reading capability to convert query into Pandas dataframe. Aim of this practice is to minimize usage of Pandas and maximize usage of SQL instead for dataprocessing and analysis.

SQLiteStudio (3.4.17) will also be used for testing and troubleshooting query prior to inserting for use by Pandas.

Reference for ideas and (only as last resort) solution:
- [SQL Challenge: E-commerce data analysis](https://www.kaggle.com/code/terencicp/sql-challenge-e-commerce-data-analysis)

In [1]:
# Import required module
import pandas as pd
import sqlite3
import os

# Connection setup

In [2]:
# Path
base:str = os.getcwd()
sqlite_path:str = os.path.join(base, "Dataset", "olist.sqlite")

# connector set up
conn = sqlite3.connect(sqlite_path)

## Testing

Testing query with CTE; previous attempt at querying result in error, testing in the beginning just in case.

In [3]:
# First layer - CTE
CTE_Query = f"""
Select *
From products p Join product_category_name_translation pcnt 
Where pcnt.product_category_name = p.product_category_name"""

output = pd.read_sql_query(CTE_Query, conn)
output
# print(CTE)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumaria,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,artes,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,esporte_lazer,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,bebes,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,utilidades_domesticas,housewares
...,...,...,...,...,...,...,...,...,...,...,...
32323,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,moveis_decoracao,furniture_decor
32324,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0,construcao_ferramentas_iluminacao,construction_tools_lights
32325,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0,cama_mesa_banho,bed_bath_table
32326,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,informatica_acessorios,computers_accessories


In [4]:
# Second Layer - Query combined with CTE
Query_2 = f"""WITH New_product_Cat AS ({CTE_Query})
Select
    npc.product_id,
    npc.product_category_name_english,
    (npc.product_length_cm 
        * npc.product_height_cm 
        * npc.product_width_cm) As vol_weight
From
    New_product_Cat npc    
    """

output = pd.read_sql_query(Query_2, conn)
output

,product_id,product_category_name_english,vol_weight
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,2240.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,10800.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,2430.0
3,cef67bcfe19066a932b7673e239eb23d,baby,2704.0
4,9dc1a7de274444849c219cff195d0b71,housewares,4420.0
...,...,...,...
32323,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor,64000.0
32324,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights,4864.0
32325,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table,5103.0
32326,83808703fc0706a22e264b9d75f04a2e,computers_accessories,8060.0


In [5]:
# Third layer - Second layer CTE query

Query_3 = f"""WITH new_product_cat AS ({Query_2})
Select
    Distinct(o.order_id),
    npc.product_category_name_english,
    npc.vol_weight,
    o.order_delivered_customer_date
From
    new_product_cat npc
Left Join order_items oi 
    USING (product_id)
Left Join orders o 
    USING (order_id)"""

print(Query_3)

output = pd.read_sql_query(Query_3, conn)
output

WITH new_product_cat AS (WITH New_product_Cat AS (
Select *
From products p Join product_category_name_translation pcnt 
Where pcnt.product_category_name = p.product_category_name)
Select
    npc.product_id,
    npc.product_category_name_english,
    (npc.product_length_cm 
        * npc.product_height_cm 
        * npc.product_width_cm) As vol_weight
From
    New_product_Cat npc    
    )
Select
    Distinct(o.order_id),
    npc.product_category_name_english,
    npc.vol_weight,
    o.order_delivered_customer_date
From
    new_product_cat npc
Left Join order_items oi 
    USING (product_id)
Left Join orders o 
    USING (order_id)


,order_id,product_category_name_english,vol_weight,order_delivered_customer_date
0,e17e4f88e31525f7deef66779844ddce,perfumery,2240.0,2018-04-27 16:42:17
1,5236307716393b7114b53ee991f36956,art,10800.0,2018-02-16 13:27:08
2,01f66e58769f84129811d43eefd187fb,sports_leisure,2430.0,2018-07-11 23:24:16
3,143d00a4f2dde4e0364ee1821577adb3,baby,2704.0,2018-08-06 15:14:32
4,86cafb8794cb99a9b1b77fc8e48fbbbb,housewares,4420.0,2018-04-13 22:41:44
...,...,...,...,...
99782,fc957026f2482ab3bddf91ebc9d0dfc5,bed_bath_table,5103.0,2018-01-23 22:58:48
99783,420937423f0cb3d3fe689330b5d385a9,computers_accessories,8060.0,2017-10-26 16:05:41
99784,59e88b7d3675e89aceaf86f372d3bc9a,computers_accessories,8060.0,2018-01-15 20:39:35
99785,6e4465d771f02e4fe335225de3c6c043,computers_accessories,8060.0,2018-03-23 20:08:40


In [6]:
# Fourth layer - Third layer CTE query

# Query_4 = 


In [7]:
# Sample query from: https://www.kaggle.com/code/terencicp/sql-challenge-e-commerce-data-analysis

ranked_categories = """
SELECT
    product_category_name_english AS category,
    SUM(price) AS sales,
    RANK() OVER (ORDER BY SUM(price) DESC) AS rank
FROM order_items
    JOIN orders USING (order_id)
    JOIN products USING (product_id)
    JOIN product_category_name_translation USING (product_category_name)
WHERE order_status = 'delivered'
GROUP BY product_category_name_english
"""

category_sales_summary = f"""
WITH RankedCategories AS (
    {ranked_categories}
)
-- Top 18 categories by sales
SELECT
    category,
    sales
FROM RankedCategories
WHERE rank <= 18
-- Other categories, aggregated
UNION ALL
SELECT
    'Other categories' AS category,
    SUM(sales) AS sales
FROM RankedCategories
WHERE rank > 18
"""

df = pd.read_sql_query(category_sales_summary, conn)
df

,category,sales
0,health_beauty,1233131.72
1,watches_gifts,1166176.98
2,bed_bath_table,1023434.76
3,sports_leisure,954852.55
4,computers_accessories,888724.61
5,furniture_decor,711927.69
6,housewares,615628.69
7,cool_stuff,610204.10
8,auto,578966.65
9,toys,471286.48


## 2. Simple Query

This section include simple query without any joining or cleaning operation.

### 2.1 Total order count
Total order count without cleaning

In [8]:
total_order_query = """
Select
    COUNT(order_id) AS vol
From orders"""

output = pd.read_sql_query(total_order_query, conn)
output

,vol
0,99441


### 2.2 Total order count (completed vs incompleted)

Total order count of completed vs incompleted.

In [26]:
total_order_query_completion = """
Select
    IIF(order_delivered_customer_date ISNULL, 
            "N", 
            "Y") 
            as Order_completion,
    COUNT(order_id) AS vol
From orders
Group By
    Order_completion"""

output = pd.read_sql_query(total_order_query_completion, conn)
output

,Order_completion,vol
0,N,2965
1,Y,96476


## 3. Intermediate Querying

### 3.3 Order vol by category

Group the count of order by category.

In [10]:
category_order = """
Select
    Distinct (pcnt.product_category_name_english) as product_list,
    Count(o.order_id) as vol
From orders o
Left Join order_items USING (order_id)
Left Join products p USING (product_id)
Left Join product_category_name_translation pcnt USING (product_category_name)
Group By
    product_list"""

output = pd.read_sql_query(category_order, conn)
output

,product_list,vol
0,None,2402
1,agro_industry_and_commerce,212
2,air_conditioning,297
3,art,209
4,arts_and_craftmanship,24
...,...,...
67,stationery,2517
68,tablets_printing_image,83
69,telephony,4545
70,toys,4117


## 2.4 Order by city

In [22]:
geo_cte = """
Select
    Distinct (gl.geolocation_zip_code_prefix) as Distinct_zip_prefix,
    *
From 
    geolocation gl
Join customers c 
    On gl.geolocation_zip_code_prefix = c.customer_zip_code_prefix
Where
    customer_city is not null
Group By
    Distinct_zip_prefix
"""

output = pd.read_sql_query(geo_cte, conn)
output

,Distinct_zip_prefix,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,1003,1003,-23.549083,-46.634864,sao paulo,SP,7ae2a9337aa4bc799723511faa1d6830,0c1a20644f0dc126c3eaff8dbc1bd12c,1003,sao paulo,SP
1,1004,1004,-23.550765,-46.635323,sao paulo,SP,a09edf8c1e842e94805a206b3d73eed5,968f6d2f674977d88a4b445a5117ccd8,1004,sao paulo,SP
2,1005,1005,-23.549980,-46.634768,sao paulo,SP,5a8b64ee6ccdae09ea823e6aa00e9517,9c84e5193d6ee59b3870e0e4e3a2dad8,1005,sao paulo,SP
3,1006,1006,-23.550696,-46.634885,sao paulo,SP,fe9db57b1fe84352125989acb12d3c68,d2d4ae284fb74a5b0f0f8af418a64b3f,1006,sao paulo,SP
4,1007,1007,-23.551083,-46.635704,sao paulo,SP,6099b4a3bbcc0a01fb20ff6cab0289e3,aa1c9c908a46198cf8b8fdc5c8037688,1007,sao paulo,SP
...,...,...,...,...,...,...,...,...,...,...,...
14832,99960,99960,-27.954114,-52.016483,charrua,RS,158af3ad0742719d373724b762396918,e96373fff9632d363f5d96e6f169ef33,99960,charrua,RS
14833,99965,99965,-28.210845,-52.059927,agua santa,RS,f93bb6c70a7306d18a24f77f1889916d,ee0a41f3ec008b1459efbd99c775e6ea,99965,agua santa,RS
14834,99970,99970,-28.348616,-51.875554,ciriaco,RS,3ab8bc00f8740d54afc4c771fb6c7f69,0528a0a940c7116ccb48fdbb8e80a8ff,99970,ciriaco,RS
14835,99980,99980,-28.418178,-51.779942,david canabarro,RS,964b34423c822e9bd9831635ccc83db2,e49eafd7e69d43b8d86f6b5590fafd02,99980,david canabarro,RS


In [25]:
# geo_cte = "Select

city_order = """
Select
    Distinct (gl.geolocation_state) as geo_state,
    Count(o.order_id) as vol
From orders o
Left Join customers c USING (customer_id)
Left Join geolocation gl On c.customer_zip_code_prefix = gl.geolocation_zip_code_prefix
Where
    geo_state is not null
Group By
    geo_state"""

output = pd.read_sql_query(city_order, conn)
output

,geo_state,vol
0,AC,7688
1,AL,34861
2,AM,5587
3,AP,4912
4,BA,365875
5,CE,63507
6,DF,93309
7,ES,316654
8,GO,133146
9,MA,53383
